In [1]:
# Importing the libraries
import sys
import py_entitymatching as em
import pandas as pd
import os

In [2]:
# Display the versions
print('python version: ' + sys.version )
print('pandas version: ' + pd.__version__ )
print('magellan version: ' + em.__version__ )

python version: 3.6.12 |Anaconda, Inc.| (default, Sep  9 2020, 00:29:25) [MSC v.1916 64 bit (AMD64)]
pandas version: 0.23.4
magellan version: 0.3.1


In [3]:
# Load csv files as dataframes and set the key attribute in the dataframe
path_A = 'music1.csv'
path_B = 'music2.csv'
A = em.read_csv_metadata(path_A, key='Sno')
B = em.read_csv_metadata(path_B, key='Sno')

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [4]:
# Display number of tuples in the datasets
print('Number of tuples in A: ' + str(len(A)))
print('Number of tuples in B: ' + str(len(B)))
print('Number of tuples in A X B (i.e the cartesian product): ' + str(len(A)*len(B)))

Number of tuples in A: 6907
Number of tuples in B: 55923
Number of tuples in A X B (i.e the cartesian product): 386260161


In [5]:
# Displaying first two entries from the first music dataset
A.head(2)

Sno                   Album_Name Album_Price Artist_Name  \
0    1  Welcome to Cam Country - EP      $4.26          Cam   
1    2                       Me 4 U      $9.99          Omi   

                                                                                             CopyRight  \
0                                                                        2015 Sony Music Entertainment   
1   2015 Ultra Records, LLC under exclusive license to Columbia Records, a Division of Sony Music E...   

   Customer_Rating                                          Genre       Price  \
0          4.72396  Country,Music,Contemporary Country,Honky Tonk      $0.99    
1          3.38158              Pop/Rock,Music,Pop,Dance,R&B/Soul  Album Only   

    Released      Song_Name  Time  
0  31-Mar-15  Runaway Train  3:01  
1        NaN       Track 14  3:41

In [6]:
# Displaying first two entries from the second music dataset
B.head(2)

Sno               Album_Name Artist_Name                      Song_Name  \
0    1  ! (Volume 2) [Explicit]       Rusko     Saxophone Stomp [Explicit]   
1    2  ! (Volume 2) [Explicit]       Rusko  I Wanna Mingle [feat. Pusher]   

   Price  Time             Released                 Label  \
0  $1.29  3:20   September 16, 2014   Decca International   
1  $1.29  2:36   September 16, 2014   Decca International   

                                                               Copyright  \
0   (C) 2014 FMLY Under Exclusive License To Universal Music Canada Inc.   
1   (C) 2014 FMLY Under Exclusive License To Universal Music Canada Inc.   

                        Genre  
0  Dance & Electronic,Dubstep  
1  Dance & Electronic,Dubstep

In [7]:
# Display the keys of the input tables
print(em.get_key(A), em.get_key(B))
# If the tables are large we can downsample the tables like this
A1, B1 = em.down_sample(A, B, 1000, 1, show_progress=False)
print("Lengths after downsampling-")
print(len(A1), len(B1))

Sno Sno
Lengths after downsampling-
557 1000


Blocking

In [8]:
# Create attribute equivalence blocker
ab = em.AttrEquivalenceBlocker()

colA = ["Sno","Album_Name","Album_Price","Artist_Name","CopyRight","Customer_Rating","Genre","Price","Released","Song_Name","Time"]
colB = ["Sno","Album_Name","Artist_Name","Song_Name","Price","Time","Released","Label","Copyright","Genre"]

# Block using Artist Name attribute
C = ab.block_tables(A1, B1, "Artist_Name","Artist_Name",
                    l_output_attrs=["Sno", "Album_Name", "Artist_Name", "CopyRight", "Released", "Song_Name" ,"Time"], 
                    r_output_attrs=["Sno", "Album_Name", "Artist_Name", "Copyright", "Released", "Song_Name", "Time"]
                    )

# Rule based blocker, which blocks on the basis of a rule
rb = em.RuleBasedBlocker()
block_f = em.get_features_for_blocking(A1, B1,validate_inferred_attr_types=False)
rule1 = ["Artist_Name_Artist_Name_lev_sim(ltuple, rtuple) > 0.8"]
rb.add_rule(rule1, block_f)
C = rb.block_tables(A1, B1, 
                    l_output_attrs=["Sno", "Album_Name", "Artist_Name", "CopyRight", "Released", "Song_Name" ,"Time"], 
                    r_output_attrs=["Sno", "Album_Name", "Artist_Name", "Copyright", "Released", "Song_Name", "Time"]
                    )

In [9]:
# Printing length of candidate set
len(C)

1498

Debugging blocker output

In [10]:
# Debug blocker output
dbg = em.debug_blocker(C, A, B, output_size=1000)

In [11]:
# Display first few tuple pairs from the debug_blocker's output
dbg.head()

_id  ltable_Sno  rtable_Sno ltable_Album_Name ltable_Artist_Name  \
0    0           5       50767    Title (Deluxe)     Meghan Trainor   
1    1           5       50797    Title (Deluxe)     Meghan Trainor   
2    2           5       50812    Title (Deluxe)     Meghan Trainor   
3    3           5       50827    Title (Deluxe)     Meghan Trainor   
4    4         155       50767    Title (Deluxe)     Meghan Trainor   

                             ltable_Genre ltable_Price ltable_Released  \
0  Pop,Music,Rock,Pop/Rock,Dance,Teen Pop       $1.29         9-Jan-15   
1  Pop,Music,Rock,Pop/Rock,Dance,Teen Pop       $1.29         9-Jan-15   
2  Pop,Music,Rock,Pop/Rock,Dance,Teen Pop       $1.29         9-Jan-15   
3  Pop,Music,Rock,Pop/Rock,Dance,Teen Pop       $1.29         9-Jan-15   
4  Pop,Music,Rock,Pop/Rock,Dance,Teen Pop       $1.29         9-Jan-15   

  ltable_Song_Name ltable_Time rtable_Album_Name rtable_Artist_Name  \
0           Credit        2:51    Title (Deluxe)     Meghan Trainor   
1           Credit        2:51    Title (Deluxe)     Meghan Trainor   
2           Credit        2:51    Title (Deluxe)     Meghan Trainor   
3           Credit        2:51    Title (Deluxe)     Meghan Trainor   
4           Credit        2:51    Title (Deluxe)     Meghan Trainor   

  rtable_Genre rtable_Price   rtable_Released rtable_Song_Name rtable_Time  
0          Pop        $1.29   January 9, 2015           Credit        2:51  
1          Pop        $1.29   January 9, 2015           Credit        2:51  
2          Pop        $1.29   January 9, 2015           Credit        2:51  
3          Pop        $1.29   January 9, 2015           Credit        2:51  
4          Pop        $1.29   January 9, 2015           Credit        2:51

Matching

In [12]:
# Sample candidate set
# S = em.sample_table(C, 538)

In [13]:
# Label S
# G = em.label_table(S, 'label')

In [14]:
# Load the pre-labeled data
path_G = 'music_labeled_data.csv'
G = em.read_csv_metadata(path_G, 
                         key='_id',
                         ltable=A, rtable=B, 
                         fk_ltable='ltable.Sno', fk_rtable='rtable.Sno')
print(len(G))

Metadata file is not present in the given path; proceeding to read the csv file.


539


In [15]:
# Split S into development set (I) and evaluation set (J)
IJ = em.split_train_test(G, train_proportion=0.7, random_state=0)
I = IJ['train']
J = IJ['test']

In [16]:
# Create a set of ML-matchers
dt = em.DTMatcher(name='DecisionTree', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')
nb = em.NBMatcher(name='NaiveBayes')

In [17]:
# Generate features
feature_table = em.get_features_for_matching(A, B, validate_inferred_attr_types=False)

In [18]:
# List the names of the features generated
feature_table['feature_name']

0                                     Sno_Sno_exm
1                                     Sno_Sno_anm
2                                Sno_Sno_lev_dist
3                                 Sno_Sno_lev_sim
4           Album_Name_Album_Name_jac_qgm_3_qgm_3
5       Album_Name_Album_Name_cos_dlm_dc0_dlm_dc0
6       Album_Name_Album_Name_jac_dlm_dc0_dlm_dc0
7                       Album_Name_Album_Name_mel
8                  Album_Name_Album_Name_lev_dist
9                   Album_Name_Album_Name_lev_sim
10                      Album_Name_Album_Name_nmw
11                       Album_Name_Album_Name_sw
12        Artist_Name_Artist_Name_jac_qgm_3_qgm_3
13    Artist_Name_Artist_Name_cos_dlm_dc0_dlm_dc0
14    Artist_Name_Artist_Name_jac_dlm_dc0_dlm_dc0
15                    Artist_Name_Artist_Name_mel
16               Artist_Name_Artist_Name_lev_dist
17                Artist_Name_Artist_Name_lev_sim
18                    Artist_Name_Artist_Name_nmw
19                     Artist_Name_Artist_Name_sw


In [19]:
# Convert the I (training set) into a set of feature vectors using F
H = em.extract_feature_vecs(I, 
                            feature_table=feature_table, 
                            attrs_after='label',
                            show_progress=False)
# Display first few rows
# H.head(3)

In [20]:
# Select the best ML matcher using cross validation
result = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, 
        exclude_attrs=['_id', 'ltable.Sno', 'rtable.Sno', 'label'],
        k=5,
        target_attr='label', metric_to_select_matcher='f1', random_state=0)
result['cv_stats']

Matcher  Average precision  Average recall  Average f1
0  DecisionTree           0.909271        0.897727    0.899879
1            RF           0.952976        0.912771    0.930205
2           SVM           0.889031        0.817641    0.846430
3        LinReg           0.929327        0.929437    0.926010
4        LogReg           0.910815        0.905628    0.907895
5    NaiveBayes           0.905036        0.944481    0.920527

In [22]:
# Manually creating features for some attribute

def time_feature(ltuple, rtuple):
    p1 = ltuple.Time
    p2 = rtuple.Time
    if p1 == p2:
        return 1.0
    else:
        return 0.0
    
def released_feature(ltuple, rtuple):
    p1 = str(ltuple.Released)
    p2 = str(rtuple.Released)
    if p1=="nan" or p2=="nan":
        return 0
    p1 = p1.split("-") # 31,mar,15
    if len(p1)<3:
        return 0
    d = {"Jan":"January","Feb":"February","Mar":"March","Apr":"April","May":"May","Jun":"June","Jul":"July","Aug":"August","Sep":"September","Oct":"October","Nov":"November","Dec":"December"}
    s1 = " " + d[p1[1]] + " " + p1[0] + ", 20" + p1[2]
    if em.lev_sim(s1,p2)>0.9:
        return 1.0
    else:
        return 0.0
    
def copyright_feature(ltuple, rtuple):
    p1 = ltuple.CopyRight
    p2 = rtuple.Copyright
    if em.lev_sim(p1,p2)>0.8:
        return 1.0
    else:
        return 0.0

def price_feature(ltuple, rtuple):
    p1 = str(ltuple.Price).strip()
    p2 = str(rtuple.Price).strip()
    if p1 == p2:
        return 1.0
    else:
        return 0.0

In [23]:
# feature_table = feature_table.drop([28,29,30,31,32,33,34,35]) 

em.add_blackbox_feature(feature_table, 'time_time_feature', time_feature)
em.add_blackbox_feature(feature_table, 'released_released_feature', released_feature)
em.add_blackbox_feature(feature_table, 'price_price_feature', price_feature)
em.add_blackbox_feature(feature_table, 'copyright_copyright_feature', copyright_feature)
H = em.extract_feature_vecs(I, feature_table=feature_table, attrs_after='label', show_progress=False)

# Select the best ML matcher using CV
result = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, 
        exclude_attrs=['_id', 'ltable.Sno', 'rtable.Sno', 'label'],
        k=5,
        target_attr='label', metric_to_select_matcher='f1', random_state=0)
result['cv_stats']

Matcher  Average precision  Average recall  Average f1
0  DecisionTree           0.955615        0.968290    0.959680
1            RF           0.962500        0.930195    0.944164
2           SVM           0.889031        0.817641    0.846430
3        LinReg           0.978333        0.949675    0.962961
4        LogReg           0.950000        0.929762    0.938331
5    NaiveBayes           0.925191        0.990909    0.955925

In [24]:
# Convert J into a set of feature vectors using feature table
L = em.extract_feature_vecs(J, feature_table=feature_table,
                            attrs_after='label', show_progress=False)

In [25]:
# Train using feature vectors from I 
rf.fit(table=H, 
       exclude_attrs=['_id', 'ltable.Sno', 'rtable.Sno', 'label'], 
       target_attr='label')

In [26]:
# Predict on L 
predictions = rf.predict(table=L, exclude_attrs=['_id', 'ltable.Sno', 'rtable.Sno', 'label'], 
              append=True, target_attr='predicted', inplace=False)

In [27]:
# Evaluate the predictions
eval_result = em.eval_matches(predictions, 'label', 'predicted')
em.print_eval_summary(eval_result)

Precision : 90.24% (37/41)
Recall : 100.0% (37/37)
F1 : 94.87%
False positives : 4 (out of 41 positive predictions)
False negatives : 0 (out of 121 negative predictions)


In [28]:
!pip install tabulate

In [29]:
precisions = [90.24,89.74,90.24,90.24,90.24,89.74,87.5,87.8,87.18,89.19]
recalls = [100.0,94.59,100.0,100.0,100.0,94.59,94.59,97.3,91.89,89.19]
f1_scores = [94.87,92.11,94.87,94.87,94.87,92.11,90.91,92.31,89.47,89.19]
x = ["None","Time","Released","Price","Copyright","Sno","Album_name","Artist_name","Genre","Song_name"]

from tabulate import tabulate

headers = ["Attribute missing","precision","recall","f1_score"]

table = []
for i in range(1,len(x)):
    p = abs(precisions[i]-precisions[0])
    r = abs(recalls[i]-recalls[0])
    f = abs(f1_scores[i]-f1_scores[0])
    table.append([x[i],p,r,f])

print("Absolute difference in values of metrics with the ones where no attribute is missing")
print(tabulate(table, headers=headers))

Absolute difference in values of metrics with the ones where no attribute is missing
Attribute missing      precision    recall    f1_score
-------------------  -----------  --------  ----------
Time                        0.5       5.41        2.76
Released                    0         0           0
Price                       0         0           0
Copyright                   0         0           0
Sno                         0.5       5.41        2.76
Album_name                  2.74      5.41        3.96
Artist_name                 2.44      2.7         2.56
Genre                       3.06      8.11        5.4
Song_name                   1.05     10.81        5.68


In [30]:
import matplotlib.pyplot as plt

d = {}
for i in range(len(x)):
    d[x[i]] = [precisions[i],recalls[i],f1_scores[i]]
    
x_axis = []
precisions = []
recalls = []
f1_scores = []
for i in sorted(d):
    x_axis.append(i)
    precisions.append(d[i][0])
    recalls.append(d[i][1])
    f1_scores.append(d[i][2])

fig, ax = plt.subplots()
plt.plot(x_axis,precisions,color='b',label="precision")
plt.plot(x_axis,recalls,color='r',label="recall")
plt.plot(x_axis,f1_scores,color='g',label="f1-score")

fig.set_size_inches(10.5, 7.5)
plt.legend(loc="upper right")
plt.title("Values of metrics when one particular attribute is missing")
plt.ylabel("Value of metric (in %)")
plt.xlabel("Attribute missing")
plt.show()